# Sentiment Analysis에 활용될 Dataset 또한 크롤링한 데이터 전처리와 같은 절차를 밟았습니다. 
# 내용은 2-1과 대부분 동일합니다.

# Importing Libraries & Df

In [2]:
# !pip install gensim

In [4]:
import pandas as pd
from datetime import datetime
import numpy as np
import scipy as sp
import sklearn
import sys
import os

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from scipy import linalg

# !pip install gensim 완료
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import pickle
import string
import time


In [5]:
path = r'C:\Users\MinSeong\Desktop\수업\비즈니스 텍마\Term Proj'
os.chdir(path)

df = pd.read_excel('5_total_train.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122130 entries, 0 to 122129
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Text    122130 non-null  object
 1   Label   122130 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


# Cleaning Text

In [4]:
def clean_text(text):
    text = re.sub('http[s]?://\S+', '', text) # http url 제거
    text = re.sub('\S*@\S*\s?', '', text) #
    text = re.sub(r'\[.*?\]', '', text) # 
    text = re.sub(r'\w*\d\w*', '', text) # 숫자 포함하는 텍스트 제거
    text = re.sub('[?.,;:|\)*~"`’!\-_+<>@\#$%&-=#}※]', ' ', text)#특수문자 이모티콘 제거.
    text = re.sub("\n", '', text) #개행문자 제거
    text = re.sub("\r", '', text) #개행문자 제거
    text = re.sub("\xa0", '', text) #개행문자 제거
    text = re.sub(r'Copyright .* rights reserved', '', text) # "Copyright all rights reserved" 제거
    return text
# content   
df['text_content'] = pd.DataFrame(df.Text.apply(lambda x: clean_text(x)))


# seperate sentences

In [5]:
import kss
# content

df['content_sep'] = df['text_content'].apply(lambda x: kss.split_sentences(x))
df['content_sep'].head()


0               [이거 정말 내 취향인데 역시 내 남친]
1                  [이거 엄청 비싼건데 너무 고마워]
2    [이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이]
3                        [이제 재수생활 끝이다]
4               [엄마 이제 어깨 펴고 다녀 나 붙었어]
Name: content_sep, dtype: object

# 띄어쓰기, 맞춤법 수정

In [6]:
def space(docu):
    docu_spaced = []
    for sent in docu:
        docu_spaced.append(spacing(sent))
    return docu_spaced

In [7]:
from pykospacing import spacing

docu = df['content_sep'].loc[0]
docu_spaced = []
for sent in docu:
    docu_spaced.append(spacing(sent))

띄어쓰기 수정

In [8]:
#띄어쓰기
from pykospacing import spacing
def space(docu):
    docu_spaced = []
    for sent in docu:
        docu_spaced.append(spacing(sent))
    return docu_spaced
    
sys.setrecursionlimit(3000)

begin = time.time()

df['content_sep']  = df['content_sep'].apply(lambda x: space(x))
df['content_sep'].head()

end = time.time()
print(np.round((end - begin)/60),"분 소요")

73.0 분 소요


맞춤법 수정

In [9]:
#맞춤법
import time
begin = time.time()


from hanspell import spell_checker
def spell(docu):
    try:
        docu_spelled = []
        for sent in docu:
            docu_spelled.append(spell_checker.check(sent).checked)
    except:
        docu_spelled = docu
    return docu_spelled
        
df['content_p'] = df['content_sep'].apply(lambda x: spell(x))

end = time.time()
print(np.round((end - begin)/60),"분 소요")

51.0 분 소요


In [10]:
# 맞춤법, 띄어쓰기 수정한 tect
#df.to_csv('preprocessed_df_pre_형태소.csv')

In [11]:
df.head()

,Text,Label,text_content,content_sep,content_p
0,이거 정말 내 취향인데 역시 내 남친,Happiness,이거 정말 내 취향인데 역시 내 남친,[이 거 정말 내 취향인데 역시 내 남친],[이거 정말 내 취향인데 역시 내 남자친구]
1,이거 엄청 비싼건데 너무 고마워,Happiness,이거 엄청 비싼건데 너무 고마워,[이 거 엄청 비싼 건데 너무 고마워],[이거 엄청 비싼 건데 너무 고마워]
2,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,Happiness,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,[이 거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이],[이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이]
3,이제 재수생활 끝이다,Happiness,이제 재수생활 끝이다,[이제 재수 생활 끝이다],[이제 재수 생활 끝이다]
4,엄마 이제 어깨 펴고 다녀 나 붙었어,Happiness,엄마 이제 어깨 펴고 다녀 나 붙었어,[엄마 이 제 어깨 펴고 다녀 나 붙었어],[엄마 이제 어깨 펴고 다녀 나붙었어]


# 숫자 제거

In [15]:
#content
p=re.compile("[0-9]+")
df['content_p'] = df['content_p'].apply(lambda x: p.sub(" ",x[0]))


IndexError: list index out of range

In [17]:

# pkl_name = 'preprocessed_SA_pre_형태소'
# with open(pkl_name,'wb') as fw:
#   pickle.dump(df,fw)

 형태소 쪼개기전 pickle loading

In [6]:
# with open('preprocessed_SA_pre_형태소','rb') as fr:
#   df = pickle.load(fr)
# df

,Text,Label,text_content,content_sep,content_p
0,이거 정말 내 취향인데 역시 내 남친,Happiness,이거 정말 내 취향인데 역시 내 남친,[이 거 정말 내 취향인데 역시 내 남친],[이거 정말 내 취향인데 역시 내 남자친구]
1,이거 엄청 비싼건데 너무 고마워,Happiness,이거 엄청 비싼건데 너무 고마워,[이 거 엄청 비싼 건데 너무 고마워],[이거 엄청 비싼 건데 너무 고마워]
2,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,Happiness,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,[이 거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이],[이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이]
3,이제 재수생활 끝이다,Happiness,이제 재수생활 끝이다,[이제 재수 생활 끝이다],[이제 재수 생활 끝이다]
4,엄마 이제 어깨 펴고 다녀 나 붙었어,Happiness,엄마 이제 어깨 펴고 다녀 나 붙었어,[엄마 이 제 어깨 펴고 다녀 나 붙었어],[엄마 이제 어깨 펴고 다녀 나붙었어]
...,...,...,...,...,...
122125,기자의 정체성이 무슨 소용이야 다 쓸모 없어 부질없어 알아주지도 않잖아,Sadness,기자의 정체성이 무슨 소용이야 다 쓸모 없어 부질없어 알아주지도 않잖아,[기자의 정체성이 무슨 소용 이야 다 쓸 모 없어 부질없어 알아주지도 않잖아],[기자의 정체성이 무슨 소용이야 다 쓸모없어 부질없어 알아주지도 않잖아]
122126,이러다가 가짜 뉴스인 거 들키기라도 하면 너는 그대로 모가지야,Angry,이러다가 가짜 뉴스인 거 들키기라도 하면 너는 그대로 모가지야,[이러다가 가짜 뉴스인 거 들 키기라도 하면 너는 그대로 모가 지야],[이러다가 가짜 뉴스 인 거 들 키기라도 하면 너는 그대로 모가 지야]
122127,누가 들킨대? 너만 입 다물고 있으면 세상 사람들 아무도 몰라,Angry,누가 들킨대 너만 입 다물고 있으면 세상 사람들 아무도 몰라,[누가 들킨 대 너만 입 다물고 있으면 세상 사람들 아무도 몰라],[누가 들킨 대 너만 입 다물고 있으면 세상 사람들 아무도 몰라]
122128,나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리한다고,Neutral,나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리한다고,[나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리...,[나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리...


# 형태소 분석

In [7]:
#!pip install konlpy
from konlpy.tag import Okt

In [8]:
with open('preprocessed_SA_pre_형태소','rb') as fr:
  df = pickle.load(fr)
df

,Text,Label,text_content,content_sep,content_p
0,이거 정말 내 취향인데 역시 내 남친,Happiness,이거 정말 내 취향인데 역시 내 남친,[이 거 정말 내 취향인데 역시 내 남친],[이거 정말 내 취향인데 역시 내 남자친구]
1,이거 엄청 비싼건데 너무 고마워,Happiness,이거 엄청 비싼건데 너무 고마워,[이 거 엄청 비싼 건데 너무 고마워],[이거 엄청 비싼 건데 너무 고마워]
2,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,Happiness,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,[이 거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이],[이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이]
3,이제 재수생활 끝이다,Happiness,이제 재수생활 끝이다,[이제 재수 생활 끝이다],[이제 재수 생활 끝이다]
4,엄마 이제 어깨 펴고 다녀 나 붙었어,Happiness,엄마 이제 어깨 펴고 다녀 나 붙었어,[엄마 이 제 어깨 펴고 다녀 나 붙었어],[엄마 이제 어깨 펴고 다녀 나붙었어]
...,...,...,...,...,...
122125,기자의 정체성이 무슨 소용이야 다 쓸모 없어 부질없어 알아주지도 않잖아,Sadness,기자의 정체성이 무슨 소용이야 다 쓸모 없어 부질없어 알아주지도 않잖아,[기자의 정체성이 무슨 소용 이야 다 쓸 모 없어 부질없어 알아주지도 않잖아],[기자의 정체성이 무슨 소용이야 다 쓸모없어 부질없어 알아주지도 않잖아]
122126,이러다가 가짜 뉴스인 거 들키기라도 하면 너는 그대로 모가지야,Angry,이러다가 가짜 뉴스인 거 들키기라도 하면 너는 그대로 모가지야,[이러다가 가짜 뉴스인 거 들 키기라도 하면 너는 그대로 모가 지야],[이러다가 가짜 뉴스 인 거 들 키기라도 하면 너는 그대로 모가 지야]
122127,누가 들킨대? 너만 입 다물고 있으면 세상 사람들 아무도 몰라,Angry,누가 들킨대 너만 입 다물고 있으면 세상 사람들 아무도 몰라,[누가 들킨 대 너만 입 다물고 있으면 세상 사람들 아무도 몰라],[누가 들킨 대 너만 입 다물고 있으면 세상 사람들 아무도 몰라]
122128,나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리한다고,Neutral,나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리한다고,[나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리...,[나는 정의의 수호자로서 꼭 올바름을 실현할 거야 기대하도록 해 정의는 언제나 승리...


In [9]:
from konlpy.tag import Okt  
okt=Okt()  

def pos_tagging(sent_list):
    sent_tokens=[]
    for sentence in sent_list:
        sent_tokens.append(okt.pos(sentence))
    return(sent_tokens)

df['content_p']=df['content_p'].apply(lambda x: pos_tagging(x))

In [10]:
df.head()

,Text,Label,text_content,content_sep,content_p
0,이거 정말 내 취향인데 역시 내 남친,Happiness,이거 정말 내 취향인데 역시 내 남친,[이 거 정말 내 취향인데 역시 내 남친],"[[(이, Determiner), (거, Noun), (정말, Noun), (내, ..."
1,이거 엄청 비싼건데 너무 고마워,Happiness,이거 엄청 비싼건데 너무 고마워,[이 거 엄청 비싼 건데 너무 고마워],"[[(이, Determiner), (거, Noun), (엄청, Adverb), (비..."
2,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,Happiness,이거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이,[이 거 내가 갖고 싶었던 건지 어떻게 알았어 정말 센스쟁이],"[[(이, Determiner), (거, Noun), (내, Noun), (가, J..."
3,이제 재수생활 끝이다,Happiness,이제 재수생활 끝이다,[이제 재수 생활 끝이다],"[[(이제, Noun), (재수, Noun), (생활, Noun), (끝, Noun..."
4,엄마 이제 어깨 펴고 다녀 나 붙었어,Happiness,엄마 이제 어깨 펴고 다녀 나 붙었어,[엄마 이 제 어깨 펴고 다녀 나 붙었어],"[[(엄마, Noun), (이제, Noun), (어깨, Noun), (펴고, Ver..."


In [11]:

# 명사, 동사, 형용사만 추출. 어미, 조사등은 배제.

def n_v_a_selection(sent_list):    
    sent_tokens=[]
    for sentence in sent_list:
        new_sent=[]
        for token in sentence:
            if (token[1]=='Noun' or token[1]=='Adjective' or token[1]=='Verb'):
                new_sent.append(token)
        sent_tokens.append(new_sent)
    return(sent_tokens)

df['content_p']=df['content_p'].apply(lambda x: n_v_a_selection(x))


In [12]:

# 표제어 추출
#pip install soylemma 
from soylemma import Lemmatizer

lemmatizer = Lemmatizer()

def lemma(sent_list):
    sent_stems=[]
    for sentence in sent_list:
        new_sent=[]
        for token in sentence:
            if (token[1]=='Adjective')|(token[1]=='Verb'): #명사는 필요 없음.
                stem_result=lemmatizer.lemmatize(token[0])
                if len(stem_result)!=0:              # 빈 리스트가 반환되는 경우 제외
                    new_sent.append(stem_result[0])  # 결과 중 맨 앞에 것만 추가
            else:
                new_sent.append(token)
                    
        sent_stems.append(new_sent)
    return(sent_stems)
            
df['content_p']=df['content_p'].apply(lambda x: lemma(x))


In [13]:
df['content_p']

0         [[(거, Noun), (정말, Noun), (내, Noun), (취향, Noun)...
1         [[(거, Noun), (비싸다, Adjective), (고맙다, Adjective)]]
2         [[(거, Noun), (내, Noun), (갖다, Verb), (싶다, Verb)...
3         [[(이제, Noun), (재수, Noun), (생활, Noun), (끝, Noun)]]
4         [[(엄마, Noun), (이제, Noun), (어깨, Noun), (펴다, Ver...
                                ...                        
122125    [[(기자, Noun), (정체, Noun), (무슨, Noun), (소용, Nou...
122126    [[(가짜, Noun), (뉴스, Noun), (인, Noun), (거, Noun)...
122127    [[(누가, Noun), (들키다, Verb), (대다, Verb), (너, Nou...
122128    [[(나, Noun), (정의, Noun), (수호, Noun), (로서, Noun...
122129    [[(이, Noun), (세상, Noun), (정의, Noun), (따위, Noun...
Name: content_p, Length: 122130, dtype: object

In [14]:
#stopwords 전에 저장
pkl_name = 'preprocessed_SA_pre_stopwords'
with open(pkl_name,'wb') as fw:
  pickle.dump(df,fw)


# stop words 지우기 check point

In [15]:

with open('preprocessed_SA_pre_stopwords','rb') as fr:
  df = pickle.load(fr)
df.reset_index(drop = True, inplace = True)

In [122]:
#df['content_p'].loc[22209]

# Stopwords 제거.



In [16]:
#분석시에는 추가
def rem_stopwords(sent_list):
    stopwords=[('이','Noun'), ('거','Noun'),('오','Noun'), ('것','Noun'),('있다','Adjective'),('있다', 'Verb'),\
               ('하다','Adjective'),('하다','Verb'),('보다','Verb'),('아니다','Adjective'),('내', 'Noun'),('안녕', 'Noun'),\
              ('요', 'Noun'),('되다', 'Verb'),('제', 'Noun'),('이다', 'Adjective'),\
              ('같다', 'Adjective'),('때', 'Noun'),('저', 'Noun'),('좀', 'Noun')]
    sent_stop=[]
    for sentence in sent_list:
        new_sent=[]
        for token in sentence:
            if token not in stopwords:
                new_sent.append(token)
        sent_stop.append(new_sent)
    return(sent_stop)
   
df['content_p']=df['content_p'].apply(lambda x: rem_stopwords(x))

In [17]:

# pos 태그 제거
def rem_tag(sent_list):
    sent_final=[]
    for sentence in sent_list:
        new_sent=[]
        for token in sentence:
            new_sent.append(token[0])
        sent_final.append(new_sent)
    return(sent_final)

#review_df['comment_n']=review_df['comment_n'].apply(lambda x: rem_tag(x))

   
df['content_p']=df['content_p'].apply(lambda x: rem_tag(x))

In [ ]:
df['content_p']

In [18]:
# one list로 만들기

def one_word_list(sent_list):
    one_list=[]
    for sentence in sent_list:
        one_list.extend(sentence)
    return(one_list)


   
df['content_p']=df['content_p'].apply(lambda x: one_word_list(x))

In [19]:
df['content_p']

0                                    [정말, 취향, 역시, 남자친구]
1                                            [비싸다, 고맙다]
2                         [갖다, 싶다, 걸다, 어떻다, 알다, 정말, 센스]
3                                       [이제, 재수, 생활, 끝]
4                                 [엄마, 이제, 어깨, 펴다, 다니다]
                              ...                      
122125                       [기자, 정체, 무슨, 소용, 알아주다, 않다]
122126               [가짜, 뉴스, 인, 들다, 키기, 너, 그대로, 모, 지다]
122127         [누가, 들키다, 대다, 너, 입, 다물다, 세상, 사람, 아무, 몰다]
122128          [나, 정의, 수호, 로서, 꼭, 올바르다, 실현, 해, 정의, 승리]
122129    [세상, 정의, 따위, 필요, 없다, 네, 입맛, 살다가다, 쫄딱, 망하다, 걸]
Name: content_p, Length: 122130, dtype: object

preprocessing 완료.

In [20]:
#df.to_csv('preprocessed_SA.csv')
with open("preprocessed_SA","wb") as fw:
    pickle.dump(df,fw)


In [22]:
with open("preprocessed_SA","rb") as fr:
    df = pickle.load(fr)